# Retrieval-Augmented Generation with Cohere language models on Amazon Bedrock and Weaviate vector database on AWS Market place

The example use case generates targeted advertisements for vacation stay listings based on a target audience. The goal is to use the user query for the target audience (e.g., “family with small children”) to retrieve the most relevant vacation stay listing (e.g., a listing with playgrounds close by) and then to generate an advertisement for the retrieved listing tailored to the target audience.

Note that the following code uses the newer `v4` Weaviate Python client, which uses gRPC under the hood and is currently in beta (as of November 2023).

This notebook should work well with the Data Science 3.0 kernel in SageMaker Studio.

## Dataset Overview
The dataset is available from [Inside AirBnB](http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2023-09-03/data/listings.csv.gz) and is licensed under a [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/).

Download the data and save it in a folder called `data`.

In [1]:
import pandas as pd
import json

# Read CSV file
csv_file = '../data/listings.csv'
df = pd.read_csv(csv_file, usecols=['host_name',
                                    'property_type',
                                    'description',
                                    'neighborhood_overview',
                                    ])

df.fillna('', inplace=True)

display(df.head())

FileNotFoundError: [Errno 2] No such file or directory: '../data/listings.csv'

## Prerequisites
To be able to follow along and use any AWS services in the following tutorial, please make sure you have an [AWS account](https://signin.aws.amazon.com/signin?redirect_uri=https%3A%2F%2Fportal.aws.amazon.com%2Fbilling%2Fsignup%2Fresume&client_id=signup).

## Step 1: Enable components of the AI-native technology stack
First, you will need to enable the relevant components discussed in the solution overview in your AWS account.
First, enable access to the Cohere Command and Embed foundation models available on Amazon Bedrock through the AWS Management Console. Navigate to the Model access page, click on Edit, and select the foundation models of your choice.

Next, set up a Weaviate cluster. First, subscribe to the [Weaviate Kubernetes Cluster on AWS Marketplace](https://aws.amazon.com/marketplace/pp/prodview-cicacyv63r43i). Then, launch the software using a [CloudFormation template according to your preferred availability zone](https://weaviate.io/developers/weaviate/installation/aws-marketplace#aws-marketplace). The CloudFormation template is pre-filled with default values. To follow along in this guide, edit the following fields:
* Stack name: Enter a stack name
* Authentication: It is recommended to enable authentication by setting helmauthenticationtype to apikey and defining a helmauthenticationapikey.
* Enabled modules: Make sure “tex2vec-aws” and “generative-aws” are present in the list of enabled modules within Weaviate.

This template takes about 30 minutes to complete.

## Step 2: Connect to Weaviate
On the SageMaker console, navigate to Notebook instances and create a new notebook instance.

Then, install the Weaviate client package with the required dependencies:

In [2]:
# pip install --pre "weaviate-client==4.*"
# pip install grpcio

Now, you can connect to your Weaviate instance with the following code. You can find the relevant information as follows:
* Weaviate URL: Access Weaviate via the load balancer URL. Go to the Services section of AWS, under EC2 > Load Balancers find the load balancer, and look for the DNS name column.
* Weaviate API Key: This is the key you set earlier in the CloudFormation template (helmauthenticationapikey).
* AWS Access Key: You can retrieve the access keys for your user in the AWS Identity and Access Management (IAM) Console.

In [35]:
import weaviate
import weaviate.classes as wvc

client = weaviate.WeaviateClient(
    weaviate.ConnectionParams.from_url("<YOUR-WEAVIATE-URL>", grpc_port=50051),  # e.g. weaviate.ConnectionParams.from_url("http://localhost:8080", grpc_port=50051)
    auth_client_secret=weaviate.AuthApiKey("<YOUR-WEAVIATE-API-KEY>"),
    additional_headers={
        "X-AWS-Access-Key": "<AWS-ACCESS-KEY>",
        "X-AWS-Access-Secret": "<AWS-ACCESS-SECRET>"
    }
)

print(client.is_ready())

client.get_meta()

True


{'hostname': 'http://[::]:8080',
 'modules': {'generative-cohere': {'documentationHref': 'https://docs.cohere.com/reference/generate',
   'name': 'Generative Search - Cohere'},
  'text2vec-cohere': {'documentationHref': 'https://docs.cohere.ai/embedding-wiki/',
   'name': 'Cohere Module'}},
 'version': '1.22.1'}

## Step 3: Configure the Amazon Bedrock module to enable Cohere models

Next, you will define a data collection (i.e., `class`) called `Listings` to store the listings’ data objects, which is analogous to creating a table in a relational database. In this step, you will configure the relevant modules to enable the usage of Cohere language models hosted on Amazon Bedrock natively from within the Weaviate vector database. The vectorizer (`"text2vec-aws"`) and generative module (` "generative-aws"`) are specified in the data collection definition. Both of these modules take three parameters:
* `"service"`: `"bedrock"` for Amazon Bedrock (Alternatively, `"sagemaker"` for Amazon Sagemaker Jumpstart)
* `"Region"`: The region where your model is deployed
* `"model"`: The foundation model’s name

In this step, you will also define the structure of the data collection by configuring its properties. Aside from the property’s name and data type, you can also configure if only the data object shall be stored or if it shall be stored together with its vector embeddings. In this example, `host_name` and `property_type` are not vectorized.

Run the following code to create the collection in your Weaviate instance.

In [36]:
client.collections.delete("Listings")

listings = client.collections.create(
    name="Listings",
    description="AirBnb Listings",
    vectorizer_config=wvc.Configure.Vectorizer.text2vec_aws(
        service="bedrock",
        region="us-east-1",
        model="cohere.TO_DO_REPLACE_ME_LATER",
        vectorize_class_name=False,
    ),
    generative_config=wvc.Configure.Generative.aws(
        service="bedrock",
        region="us-east-1",
        model="cohere.command-text-v14",
    ),
    properties=[
        wvc.Property(
            name="description",
            data_type=wvc.DataType.TEXT,
            description="The description of the apartment listing",
        ),
        wvc.Property(
            name="host_name",
            data_type=wvc.DataType.TEXT,
            description="The name of the host of the apartment listing",
            skip_vectorization=True,
            vectorize_property_name=False
        ),
        wvc.Property(
            name="neighborhood_overview",
            data_type=wvc.DataType.TEXT,
            description="The description of  the neighbourhood of the apartment listing",
        ),
        wvc.Property(
            name="property_type",
            data_type=wvc.DataType.TEXT,
            description="The type of property of the listing",
            skip_vectorization=True,
            vectorize_property_name=False
        )
    ]
)

listings = client.collections.get("Listings")
print(listings.config)

## Step 4: Ingest data into the Weaviate vector database

You can now add objects to Weaviate. You will be using a batch import process for maximum efficiency. Run the code below to import data. During the import, Weaviate will use the defined vectorizer to create a vector embedding for each object. The following code loads objects initializes a batch process, and adds objects to the target collection one by one.

In [37]:
listings_to_add = [
    {
        "description": row["description"],
        "host_name": row["host_name"],
        "neighborhood_overview": row["neighborhood_overview"],
        "property_type": row["property_type"],
    } for _, row in df.iterrows()
]

response = listings.data.insert_many(listings_to_add)
print(response)

BatchObjectReturn(all_responses=[UUID('49556890-16d1-47fd-8bd5-435ae1172aa2'), UUID('4abc994f-5690-46a9-bfe4-32cc5a99d67e'), UUID('262faf04-9aba-458b-8be4-cee6e4196818'), UUID('22cdc189-217b-4118-8a31-ccdea86cbe01'), UUID('38856761-d2ce-4cd5-8ca7-f39533a0c275'), UUID('d51520b0-edc7-4055-9ca1-b27852b9dc7e'), UUID('ab733713-b9cf-4e59-8f69-5e8541c411f6'), UUID('c1112ab7-888a-4dd6-bdfd-c30714508b5a'), UUID('93250be6-b48c-4cba-b8c9-ad332102ad9c'), UUID('e9e8dc0c-1fe3-4d64-a292-6970f593a0fb'), UUID('af75ce8c-001a-40ba-ae2b-d93391e4adb9'), UUID('2bbe11ac-7255-41cd-926d-9be5e536f4a4'), UUID('86fece16-402f-479b-9389-3be7b1f49c0e'), UUID('40c0718c-3862-4d6b-a979-d92a36817a0d'), UUID('80f9f781-d9fa-4b95-aa2b-544f31cbf10a'), UUID('004a3381-c173-46eb-bb4c-14cd57782444'), UUID('16c8ed2a-f663-430e-b0ed-11611ed97203'), UUID('528685f0-0764-4d85-a3a4-5a9fe8e49aee'), UUID('7a49f17f-a334-4fa7-8a4a-239148ee3c02'), UUID('d931e593-b011-4aa0-ad66-dcd003497fc3'), UUID('fea3b80f-7cab-49fe-aaf4-83ca6b19c7dd'), U

## Step 5: Retrieval-Augmented Generation to generate targeted advertisements

Finally, you can build a RAG pipeline by implementing a generative search query on your Weaviate instance. For this, you will first define a prompt template in the form of an f-string that can take in the user query (`{target_audience}`) directly and the additional context (`{{host_name}}`, `{{property_type}}`, `{{description}}`, `{{neighborhood_overview}}`) from the vector database at runtime.

Next, you will run a generative search query. This prompts the defined generative model with a prompt that is comprised of the user query as well as the retrieved data. The following query retrieves one listing object (`.with_limit(1)`) from the `Listings` collection that is most similar to the user query (`.with_near_text({"concepts": target_audience})`). Then the user query (`target_audience`) and the retrieved listings properties (`["description", "neighborhood", "host_name", "property_type"]`) are fed into the prompt template.

In [40]:
def generate_targeted_ad(target_audience):
    generate_prompt = f"""You are a copywriter.
    Write short advertisement for the following vacation stay.
    Host: {{host_name}}
    Property type: {{property_type}}
    Description: {{description}}
    Neighborhood: {{neighborhood_overview}}
    Target audience: {target_audience}
    """

    result = listings.generate.near_text(
        query=target_audience,
        limit=1,
        single_prompt=generate_prompt
    )

    return result

Below, you can see that the results for the `target_audience = “Family with small children”`.

In [48]:
result = generate_targeted_ad("Families with young children")
print(result.objects[0].generated)


Looking for a family-friendly home away from home in the heart of Amsterdam? Look no further than this charming townhouse in the Helmersbuurt neighborhood! With a spacious layout spread across two floors, this 120ft² home is the perfect size for a family of four or group of friends. The ground floor features a convenient toilet, a cozy living room with a fireplace, and a fully-equipped kitchen with a large dining table. Head upstairs to find the master bedroom with a double bed and three children's rooms - two with double beds and one with a single bed. This home is perfect for kids, but adults will love it too! You'll be within walking distance of the Vondelpark, the museum square, and some of the best bars, restaurants, and cafes in the city. Plus, with a tram stop nearby, you'll have easy access to even more of Amsterdam's top attractions. Come experience the best of the Netherlands in this cozy and convenient home! 

# Your Home in Amsterdam

*Entire Townhouse*

Child-Friendly

12

Here is another example for an elderly couple.

In [49]:
result = generate_targeted_ad("Elderly couple")
print(result.objects[0].generated)


Looking for a quiet getaway in one of the most beautiful and historic cities in the world? Look no further than this charming private room in the heart of Amsterdam! With stunning canal views and a prime location in a renovated 1930 canal house, you'll be able to enjoy the best of what the city has to offer without sacrificing comfort or convenience. 

The room itself is spacious and well-appointed, with a private shower and toilet, TV, free Wi-Fi, and a king-size bed. You'll have access to a fridge, coffee, and tea facilities, making it easy to relax and feel at home after a long day of exploring the city. 

With weekly and monthly rates available, this is the perfect spot for an extended stay in Amsterdam. And while the city center can be bustling, you'll be able to escape the crowds and enjoy a peaceful retreat in this quiet and cozy room. 

So whether you're looking for a romantic getaway or a relaxing vacation, don't miss out on the opportunity to stay in this stunning private ro